In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import astropy
from astropy.units import au, minute

In [2]:
# Local imports
import kepler_sieve
from asteroid_spline import get_df_shape, spline_ast_data
from asteroid_direction import calc_dir_ast2obs
from astro_utils import deg2dist
from db_utils import sp2df, df2db

In [3]:
# Column groups - detections
cols_u_obs = ['uObs_x', 'uObs_y', 'uObs_z']
cols_q_obs = ['qObs_x', 'qObs_y', 'qObs_z']

# Column groups - asteroid
cols_u_ast = ['uAst_x', 'uAst_y', 'uAst_z']
cols_spline_ast = cols_u_ast + ['LightTime']

In [4]:
# Get detection times
sp_name = 'KS.GetDetectionTimes'
params = dict()
dt = sp2df(sp_name=sp_name, params=params)

In [5]:
# Calculate range of times required for splines
pad = 16.0
mjd0 = np.floor(np.min(dt.mjd)) - pad
mjd1 = np.ceil(np.max(dt.mjd)) + pad

print(f'Date range for splines: {mjd0}-{mjd1}')

Date range for splines: 58254.0-59312.0


In [19]:
# Get block of asteroid detections (directions only)
sp_name = 'KS.GetDetectionDirections'
params = {
    'd0': 901,
    'd1': 905,
}
det = sp2df(sp_name=sp_name, params=params)

# Rename columns
col_tbl_obs = {
    'mjd': 't_obs',
    'ux': 'uObs_x',
    'uy': 'uObs_y',
    'uz': 'uObs_z',
}
det.rename(columns=col_tbl_obs, inplace=True)

# Set index to be DetectionID
det.set_index(keys='DetectionID', drop=False, inplace=True)

In [20]:
det

,DetectionID,t_obs,uObs_x,uObs_y,uObs_z,qObs_x,qObs_y,qObs_z
DetectionID,,,,,,,,
901,901,58270.168125,-0.914316,-0.028198,0.404018,-0.342260,-0.947314,-0.000032
902,902,58270.168125,-0.911795,-0.021422,0.410086,-0.342260,-0.947314,-0.000032
903,903,58270.169965,-0.941895,-0.058932,0.330698,-0.342231,-0.947325,-0.000032
904,904,58270.169965,-0.941838,-0.058847,0.330874,-0.342231,-0.947325,-0.000032


In [21]:
# Extract array of detection times
t_obs = det.t_obs.values

In [22]:
# Get asteroid directions in this time range
sp_name = 'KS.GetAsteroidDirections'
params = {
    'n0': 0,
    'n1': 3,
    'mjd0': mjd0,
    'mjd1': mjd1
}
ast_in = sp2df(sp_name=sp_name, params=params)

# Rename columns
col_tbl_ast = {
    'ux': 'uAst_x',
    'uy': 'uAst_y',
    'uz': 'uAst_z',
}
ast_in.rename(columns=col_tbl_ast, inplace=True)

In [23]:
# Shape of asteroid directions used in spline
N_ast, N_t_in = get_df_shape(df=ast_in, id_col='AsteroidID')

In [24]:
# Data shape
print('Shape of input data:')
N_det = t_det.shape[0]
print(f'N_ast = {N_ast}')
print(f'N_det = {N_det}')
# print(f'N_t_in = {N_t_in}')

Shape of input data:
N_ast = 2
N_det = 4


In [25]:
ast_in

,AsteroidID,TimeID,mjd,uAst_x,uAst_y,uAst_z,LightTime
0,1,83888640,58256.0,-0.695441,0.695787,0.179561,21.312345
1,1,83894400,58260.0,-0.710843,0.680749,0.176872,21.715483
2,1,83900160,58264.0,-0.726426,0.664782,0.174268,22.114047
3,1,83905920,58268.0,-0.742106,0.647905,0.171747,22.507469
4,1,83911680,58272.0,-0.757805,0.630133,0.169305,22.895268
...,...,...,...,...,...,...,...
525,2,85386240,59296.0,0.876943,-0.416402,0.239959,34.756723
526,2,85392000,59300.0,0.886408,-0.397316,0.237530,34.511296
527,2,85397760,59304.0,0.895327,-0.378259,0.235178,34.245183
528,2,85403520,59308.0,0.903706,-0.359268,0.232898,33.958757


In [58]:
# Spline asteroid directions to match these times
ast = spline_ast_data(df_ast=ast_in, ts=t_det, cols_spline=cols_spline_ast)

# Add  column for the DetectionID
ast['DetectionID'] = np.tile(det.DetectionID.values, N_ast)

# Rename column from mjd to t_ast
ast.rename(columns={'mjd':'t_ast'}, inplace=True)

# Reorder columns
cols_ast = ['AsteroidID', 'DetectionID', 't_ast'] + cols_spline_ast
ast = ast.reindex(columns=cols_ast)

In [59]:
ast

,AsteroidID,DetectionID,t_ast,uAst_x,uAst_y,uAst_z,LightTime
0,1,901,58270.168125,-0.750618,0.638382,0.170414,22.718397
1,1,902,58270.168125,-0.750618,0.638382,0.170414,22.718397
2,1,903,58270.169965,-0.750625,0.638374,0.170413,22.718575
3,1,904,58270.169965,-0.750625,0.638374,0.170413,22.718575
4,2,901,58270.168125,-0.096585,0.916142,-0.389044,24.277859
5,2,902,58270.168125,-0.096585,0.916142,-0.389044,24.277859
6,2,903,58270.169965,-0.096603,0.916141,-0.389041,24.277919
7,2,904,58270.169965,-0.096603,0.916141,-0.389041,24.277919


In [26]:
# Extract detection_id and asteroid_id
detection_id = ast.DetectionID.values
asteroid_id = ast.AsteroidID.values

In [29]:
asteroid_id

array([1, 1, 1, 1, 2, 2, 2, 2])

In [28]:
detection_id

array([901, 902, 903, 904, 901, 902, 903, 904])

In [34]:
# Direction of detections
u_obs = det[cols_u_obs].values.reshape((1, N_det, 3))

# Observer position for detections
q_obs = det[cols_q_obs].values.reshape((1, N_det, 3))

# Direction of asteroids to detection times
u_ast = ast[cols_u_ast].values.reshape((N_ast, N_det, 3))

In [35]:
# Review shapes
print(f'u_obs.shape = {u_obs.shape}')
print(f'q_obs.shape = {q_obs.shape}')
print(f'u_ast.shape = {u_ast.shape}')

u_obs.shape = (1, 4, 3)
q_obs.shape = (1, 4, 3)
u_ast.shape = (2, 4, 3)


In [36]:
u_obs

array([[[-0.91431619, -0.02819766,  0.40401831],
        [-0.91179519, -0.02142212,  0.41008613],
        [-0.94189491, -0.05893232,  0.33069768],
        [-0.94183819, -0.05884734,  0.33087431]]])

In [37]:
u_ast

array([[[-0.75061787,  0.63838231,  0.17041367],
        [-0.75061787,  0.63838231,  0.17041367],
        [-0.75062509,  0.63837412,  0.17041255],
        [-0.75062509,  0.63837412,  0.17041255]],

       [[-0.09658516,  0.91614203, -0.38904377],
        [-0.09658516,  0.91614203, -0.38904377],
        [-0.09660262,  0.91614146, -0.38904078],
        [-0.09660262,  0.91614146, -0.38904078]]])

In [39]:
# The squared cartesian distance
s2 = np.sum(np.square(u_ast - u_obs), axis=2)

In [40]:
# Review shape
print(f's2.shape = {s2.shape}')

s2.shape = (2, 4)


In [41]:
# Threshold: 2 arc minutes
# arcmin_max: float = 2.0
arcmin_max: float = 60*60
s_max: float = deg2dist(arcmin_max / 60.0)
s2_max = np.square(s_max)

print('Threshold distance:')
print(f'arcmin_max: {arcmin_max:8}')
print(f's_max:      {s_max:5.2e}')

Threshold distance:
arcmin_max:     3600
s_max:      1.00e+00


In [60]:
# Build the mask
mask_2d = (s2 < s2_max)
mask = mask_2d.flatten()

In [61]:
# Arrays of near interactions only
detection_id_near = detection_id[mask]
asteroid_id_near = asteroid_id[mask]
s_near = np.sqrt(s2[mask_2d])

In [54]:
# Look up detection data at near interactions only
t_obs_near = det.loc[detection_id_near, 't_obs'].values
u_obs_near = det.loc[detection_id_near, cols_u_obs].values
q_obs_near = det.loc[detection_id_near, cols_q_obs].values

In [56]:
# Copy splined asteroids near detections
dna = ast[mask].copy()

# Add columns for detections near asteroids
dna['t_obs'] = t_obs_near
dna['u_obs'] = u_obs_near
dna['q_obs'] = q_obs_near

# Reorder columns
cols_dna = ['AsteroidID', 'DetectionID', 't_obs'] + cols_u_obs + cols_q_obs + cols_u_ast
# dna = dna.reindex(columns=cols_dna)

In [57]:
dna

,AsteroidID,DetectionID,t_ast,uAst_x,uAst_y,uAst_z,t_obs
0,1,901,58270.168125,-0.750618,0.638382,0.170414,58270.168125
1,1,902,58270.168125,-0.750618,0.638382,0.170414,58270.168125
2,1,903,58270.169965,-0.750625,0.638374,0.170413,58270.169965
3,1,904,58270.169965,-0.750625,0.638374,0.170413,58270.169965
